In [ ]:
import os
import sys
import numpy as np
from scipy.signal import lti, lsim, welch
if not '..' in sys.path:
    sys.path.append('..')
from pfcommon import OU_2

DEBUG = False
db = 10

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, NullLocator, FixedFormatter
import seaborn as sns

fontsize = 9
lw = 0.75
matplotlib.rc('font', **{'family': 'Times', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

Load the data:

In [ ]:
N = 100
data_file = os.path.join('..','data','Sardinia','SM_configs_from_data','001',
                         'V2020_Rete_Sardegna_2021_06_03cr_AC_TF_-6.0_2.0_{}.npz'.format(N))
data = np.load(data_file, allow_pickle=True)
F = data['F']

In [ ]:
if DEBUG:
    F0 = 1
    Z,P,K = np.array([]),-2*np.pi*np.array([F0]),0.01*2*np.pi*F0
    csys = lti(Z,P,K)
    _,tf = csys.freqresp(2*np.pi*F)
else:
    TF = data['TF']
    OUT = data['OUT']
    var_names = data['var_names']
    var_type = 'bus_ur'
    if var_type == 'gen':
        gen_name = 'Grid-CODCTI-CODCTI0201________SUBNET__-CODCTI0201GGR1____GEN_____.ElmSym.speed'
        idx = np.where(var_names==gen_name)[0][0]
    elif var_type == 'bus_fe':
        bus_name = 'Grid-BNFC_I-BNFC_I0601________SUBNET__-BNFC_I0601A1______BUS_____.ElmTerm.fe'
        idx = np.where(var_names==bus_name)[0][0]
    elif var_type == 'bus_ur':
#         bus_name = 'Grid-BNFC_I-BNFC_I0601________SUBNET__-BNFC_I0601A1______BUS_____.ElmTerm.ur'
        bus_name = 'Grid-CODCTI-CODCTI3801________SUBNET__-CODCTI3801B1______BUS_____.ElmTerm.ur'
        idx = np.where(var_names==bus_name)[0][0]
    else:
        raise Exception(f"Unknown variable type '{var_type}'")
    tf = TF[0,:,idx]
    out = OUT[0,:,idx]

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(5,3))
ax.semilogx(F, db*np.log10(np.abs(tf)), 'k', lw=1.5)
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('PSD [dB]')
sns.despine()
fig.tight_layout()

Performs the vector fitting for a given number of poles:

In [ ]:
def run_vf(X, F, n_poles, n_iter=3, weights=None, do_plot=False):
    Y = X.astype(np.complex128)
    if weights is None:
        weights = np.ones(F.size, dtype=np.float64)
    else:
        assert weights.size == F.size
        weights = weights.astype(np.float64)

    F0,F1 = np.log10(F[[0,-1]])
    s = (2j*np.pi*F).astype(np.complex128)

    import vectfit3 as vf
    opts = vf.opts.copy()
    opts['asymp'] = 2
    opts['skip_res'] = True  # skip residue computation
    opts['spy2'] = False     # do not plot the results

    # initial guess for pole positions
    poles = -2*np.pi*np.logspace(F0, F1, n, dtype=np.complex128)
    for i in range(n_iter):
        if i == n_iter-1:
            opts['skip_res'] = False
            opts['spy2'] = do_plot
        SER,poles,rmserr,fit = vf.vectfit(Y, s, poles, weights, opts)
    return SER,poles,rmserr,fit

Choose the number of poles based on a threshold the RMS error:

In [ ]:
if not DEBUG:
    n_poles = np.r_[1:51]
    rms_thresh = 10 ** (np.floor(np.log10(np.abs(tf).mean())) - 3)
    for n in n_poles:
        SER,poles,rmserr,fit = run_vf(tf, F, n)
        if abs(rmserr) < rms_thresh:
            break
    print('Number of poles sufficient to have an RMS error below {:g}: {}.'.format(rms_thresh, n))

The results of the vector fitting:

In [ ]:
if not DEBUG:
    SER,poles,rmserr,fit = run_vf(tf, F, n, do_plot=True)
    csys = lti(SER['A'],SER['B'],SER['C'],SER['D'])

Load information:

In [ ]:
load_name = 'EqX_BNFC_I0601TRR_____LOAD____'
PF = data['PF'].item()
P = PF['loads'][load_name]['P'] # [MW]

Generate an OU process with the appropriate statistics:

In [ ]:
tend = 10800
srate = 200
dt = 1/srate
mean,stddev,tau = 0,0.1*P,20e-3
μ,c,α = mean,stddev*np.sqrt(2/tau),1/tau
cutoff = α/(2*np.pi)
time = np.r_[0 : tend+dt/2 : dt]
N = time.size
U = OU_2(dt, α, μ, c, N)

Filter the OU process with the TF extracted above:

In [ ]:
_,Y,X = lsim(csys, U, time)
assert Y.imag.max() < 1e-10
Y = Y.real

Plot the OU process and the corresponding filtered signal:

In [ ]:
ds = 100
fig,ax = plt.subplots(1, 1, figsize=(5,3))
ax.plot(time[::ds], U[::ds], 'k', lw=0.5, label='U')
ax.plot(time[::ds], Y[::ds], 'r', lw=0.5, label='Y')
ax.set_xlabel('Time [s]')
ax.legend(loc='upper left')
sns.despine()
fig.tight_layout()

Compute the PSDs:

In [ ]:
window = 200 / dt
onesided = True
def run_welch(x, dt, window, onesided):
    freq,P = welch(x, 1/dt, window='hamming',
                   nperseg=window, noverlap=window/2,
                   return_onesided=onesided, scaling='density')
    if onesided:
        P /= 2
    else:
        Nf = freq.size
        freq = freq[:Nf//2]
        P = P[:Nf//2]
    return freq, P, np.sqrt(P)

freq,P_U,abs_U = run_welch(U, dt, window, onesided)
P_U_db = db*np.log10(P_U)
abs_U_db = db*np.log10(abs_U)

freq,P_Y,abs_Y = run_welch(Y, dt, window, onesided)
P_Y_db = db*np.log10(P_Y)
abs_Y_db = db*np.log10(abs_Y)

P_U_theor = (c/α)**2 / (1 + (2*np.pi*F/α)**2)
P_U_theor_db = db*np.log10(P_U_theor)
abs_U_theor = np.sqrt(P_U_theor)
abs_U_theor_db = db*np.log10(abs_U_theor)

abs_TFxU = np.abs(tf)*abs_U_theor
P_TFxU = abs_TFxU**2
abs_TFxU_db = db*np.log10(abs_TFxU)
P_TFxU_db = db*np.log10(P_TFxU)

Plot the PSDs:

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(8,4), sharex=True)
# ax[0].plot(freq, abs_U_db, 'k', lw=0.75, label='U')
# ax[0].plot(freq, abs_U_theor_db, color=[1,0,1], lw=2, label='U theor')
ax[0].plot(freq, abs_U, 'k', lw=0.75, label='U')
ax[0].plot(F, abs_U_theor, color=[1,0,1], lw=2, label='U theor')
ax[0].plot(cutoff+np.zeros(2), ax[0].get_ylim(), '--', color=[0,1,0], lw=1, label='OU cutoff')
# ax[1].plot(freq, abs_Y_db, 'r', lw=0.75, label='Y')
# ax[1].plot(F, db*np.log10(np.abs(tf)), 'b', lw=1, label='TF')
# ax[1].plot(freq, abs_U, 'k', lw=0.75, label='U')
ax[1].plot(freq, abs_Y, 'r', lw=0.75, label='Y')
ax[1].plot(F, np.abs(tf), 'b', lw=1, label='TF')
ax[1].plot(F, np.abs(tf)*abs_U_theor, 'g', lw=1, label='TF*IN')
if not DEBUG:
    ax[1].plot(F, np.abs(out), 'm--', lw=1, label='OUT')
# ax[1].set_ylim([-70, 10])
for a in ax:
    a.legend(loc='lower left', frameon=False)
    a.set_xscale('log')
    a.set_xlabel('Frequency [Hz]')
# ax[0].set_ylabel('PSD')
sns.despine()
fig.tight_layout()

In [ ]:
gray = 0.7 + np.zeros(3)
fig,ax = plt.subplots(3, 1, figsize=(3.5,4.5), sharex=True)
ax[0].plot(freq[1:], P_U_db[1:], color=gray, lw=0.75, label='OU')
ax[0].plot(F, P_U_theor_db, 'k', lw=2, label='OU theor.')
ax[0].plot(cutoff+np.zeros(2), ax[0].get_ylim(), '--', color='k',
           lw=1, label='OU cutoff')
ax[1].plot(F, db*np.log10(np.abs(tf)), 'k', lw=2, label='TF')
ax[1].plot(F, db*np.log10(np.abs(np.squeeze(fit))), color=gray,
           lw=1, label=f'Fit (# poles = {poles.size})')
ax[2].plot(freq, P_Y_db, color=gray, lw=1, label='OUT')
ax[2].plot(F, P_TFxU_db, 'k', lw=1, label='TF x OU theor.')

ax[0].set_ylabel('PSD')
ax[1].set_ylabel(r'|Y(j$\omega$)| [dB{}]'.format(db))
ax[2].set_ylabel('PSD')
ticks = np.logspace(-3,2,6)
for a in ax:
    a.xaxis.set_major_locator(FixedLocator(ticks))
    a.xaxis.set_minor_locator(NullLocator())
    a.set_xscale('log')
    a.grid(which='major', axis='x', ls=':', lw=0.5, color=[.6,.6,.6])
ax[0].legend(loc='lower left', frameon=False, fontsize=8)
ax[1].legend(loc='best', bbox_to_anchor=(0.55, 0.6, 0.5, 0.5), frameon=False, fontsize=8)
ax[2].legend(loc='lower left', frameon=False, fontsize=8)
ax[-1].set_xlim(ticks[[0,-1]])
# ax[-1].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
ax[-1].set_xlabel('Frequency [Hz]')
sns.despine()
fig.tight_layout()
plt.savefig(f'spectra_input_output_{var_type}.pdf')